# 讀取開信行為資料

recipientSn 寄信行為<br>
campaignSn 活動編號<br>
mailAddress 收件人email<br>
domain 收件eamil的domain<br>
openedFlag 開信行為<br>

In [1]:
import pandas as pd
import re
import os
import jieba
import jieba.posseg as pseg
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [2]:
## turn back to main directory
os.chdir("../")
os.getcwd()

'/project/at082-group17'

In [26]:
fileName = '201903'

## 篩選要讀取的欄位

In [27]:
# 取出col欄位
useCols = list(pd.read_csv('data/2018_1_8/' + fileName + '.csv', nrows =1))
useCols.remove('subject')
useCols.remove('fromName')
print(useCols)

['recipientSn', 'campaignSn', 'mailAddress', 'domain', 'openedFlag', 'bouncedFlag', 'fromMailAddress', 'scheduledDate']


## 讀檔！！！ 這很久

In [28]:
openMailData = pd.read_csv('data/2018_1_8/' + fileName + '.csv', usecols = useCols, parse_dates = ['scheduledDate'])

In [29]:
openMailData.head()

,recipientSn,campaignSn,mailAddress,domain,openedFlag,bouncedFlag,fromMailAddress,scheduledDate
0,0000000068d9fa81016934e0638e22dc,000000006911118b016934deb6555b88,72ecb4a0c0d39dc412592b9a59614d203bcd18f87fa512...,hotmail.com,NaN,NaN,vividomi0v0@gmail.com,2019-03-01 00:11:51
1,0000000068d9fa81016934e064da22dd,000000006911118b016934deb6555b88,ce593074723f55202b89b8e5521b7c596f8a8320c67ff1...,gmail.com,NaN,NaN,vividomi0v0@gmail.com,2019-03-01 00:11:51
2,0000000068d9fa81016934e064da22de,000000006911118b016934deb6555b88,2bec838f1574135bcadb3290b6d8b401ed011b4b7e291c...,gmail.com,NaN,NaN,vividomi0v0@gmail.com,2019-03-01 00:11:51
3,0000000068d9fa81016934e064da22df,000000006911118b016934deb6555b88,9f54e3b717ba81ad300b51814162322db4603737694a70...,firstlaw.com,Y,NaN,vividomi0v0@gmail.com,2019-03-01 00:11:51
4,0000000068d9fa81016934e064da22e0,000000006911118b016934deb6555b88,7498256384788f94c6fe23148a4cdf7b764c24837d40bf...,gmail.com,NaN,NaN,vividomi0v0@gmail.com,2019-03-01 00:11:51


## 將被退信的資料移除 並刪除’bouncedFlag‘欄位

In [30]:
openMailDataIndex = openMailData['bouncedFlag'] != 'Y'
openMailData = openMailData[openMailDataIndex]
del openMailData['bouncedFlag']

## 將openedFlag 轉bool

In [31]:
openMailData['openedFlag'] = openMailData['openedFlag'] == 'Y'

In [32]:
openMailData.describe()

,recipientSn,campaignSn,mailAddress,domain,openedFlag,fromMailAddress,scheduledDate
count,10315342,10315342,10315342,10315342,10315342,10315342,10315342
unique,10315342,2447,4040870,84548,2,559,2188
top,0000000068d9fa810169ad1646654b96,000000006911118b01693322454c0fff,be952b5247541f396c73a250a58dba74e8fa2079c3ce50...,gmail.com,False,tgo@thsrc.com.tw,2019-03-04 11:30:00
freq,1,335587,75,4825825,8676205,1654162,335587
first,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-01 00:11:51
last,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-31 23:06:16


## Features Combination by groupby(對domain做處理)

In [33]:
df_temp = openMailData[["domain", "scheduledDate"]].groupby("domain").count().reset_index()
df_temp.columns = ["domain", "domain_count"]
openMailData = openMailData.merge(df_temp, how="left", on="domain")
openMailData.head()

,recipientSn,campaignSn,mailAddress,domain,openedFlag,fromMailAddress,scheduledDate,domain_count
0,0000000068d9fa81016934e0638e22dc,000000006911118b016934deb6555b88,72ecb4a0c0d39dc412592b9a59614d203bcd18f87fa512...,hotmail.com,False,vividomi0v0@gmail.com,2019-03-01 00:11:51,1050263
1,0000000068d9fa81016934e064da22dd,000000006911118b016934deb6555b88,ce593074723f55202b89b8e5521b7c596f8a8320c67ff1...,gmail.com,False,vividomi0v0@gmail.com,2019-03-01 00:11:51,4825825
2,0000000068d9fa81016934e064da22de,000000006911118b016934deb6555b88,2bec838f1574135bcadb3290b6d8b401ed011b4b7e291c...,gmail.com,False,vividomi0v0@gmail.com,2019-03-01 00:11:51,4825825
3,0000000068d9fa81016934e064da22df,000000006911118b016934deb6555b88,9f54e3b717ba81ad300b51814162322db4603737694a70...,firstlaw.com,True,vividomi0v0@gmail.com,2019-03-01 00:11:51,3
4,0000000068d9fa81016934e064da22e0,000000006911118b016934deb6555b88,7498256384788f94c6fe23148a4cdf7b764c24837d40bf...,gmail.com,False,vividomi0v0@gmail.com,2019-03-01 00:11:51,4825825


## 對時間做處理(轉星期,與時間取cos)

In [34]:
openMailData['dayofweek'] = openMailData['scheduledDate'].apply(lambda x: x.dayofweek)

In [35]:
import math
def myCos(x):
    return math.cos(x * math.pi)
def getTime(x):
    time = (x.second + (x.minute * 60) + (x.hour  * 60 * 60)) / (60 * 60 * 12)
    return myCos(time)
openMailData['time'] = openMailData['scheduledDate'].apply(getTime)

## 將mail的資料 存為pickle檔

In [36]:
## save data as pickle format
with open('data/pickle/' + fileName + '.p', "wb") as file:
    pickle.dump(openMailData, file)

## 讀取回pickle 看是否正常

In [39]:
## load 'article_cutted'
with open('data/pickle/' + fileName + '.p', "rb") as file:
    openMailData = pickle.load(file)

In [15]:
openMailData.head()

,recipientSn,campaignSn,mailAddress,domain,openedFlag,fromMailAddress,scheduledDate,domain_count,dayofweek,time
0,00000000606f8cfb0160af1258331bb9,00000000609630da0160af09b8626299,6a359cb4196db9a02c8d195b73f11374d65591365a37d8...,chwa.com.tw,True,jerry.org101@gmail.com,2018-01-01 08:14:59,156,0,-0.555510
1,00000000606f8cfb0160af1258891bba,00000000609630da0160af09b8626299,804fdb3632b9c90532fb1abaa5945a8d869ca05b92e75a...,gmail.com,False,jerry.org101@gmail.com,2018-01-01 08:14:59,2943558,0,-0.555510
2,00000000606f8cfb0160af1258891bbb,00000000609630da0160af09b8626299,fa9ae8234796d6caf56919a79af59276f976e928bb65b5...,jerry.org.tw,True,jerry.org101@gmail.com,2018-01-01 08:14:59,25,0,-0.555510
3,00000000606f8cfb0160af1258891bbc,00000000609630da0160af09b8626299,f58cab34ed3a0687423ecb6a22aba1a7cad758370428ea...,jerry.org.tw,False,jerry.org101@gmail.com,2018-01-01 08:14:59,25,0,-0.555510
4,00000000606f8cfb0160af3c39a61c20,00000000609630da0160a1fa993b43fe,3f62411f0d85adbcc6010c8ca98fae7e6165f481175588...,msa.hinet.net,False,service.edm@hotains.com.tw,2018-01-01 09:00:00,99925,0,-0.707107
